In [1]:
import pandas as pd
from experiments.models.GPT.GPTModel import GPTInference
from tqdm import tqdm
import re, string
import numpy as np
import math
from tqdm import tqdm 

/home/ramprasad.sa/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from experiments.models.Mistral import MistralInference
mistral_model = MistralInference()

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.67s/it]


In [3]:
df_llm = pd.read_csv('/home/ramprasad.sa/factual_evaluation_source_based/annotations/round1_aggrefact/dialoguefact_two_annotators_round1.csv')
df_llm = df_llm[df_llm['user_id'] == 'ann_wjbp']
df_sota = pd.read_csv('/home/ramprasad.sa/factual_evaluation_source_based/datasets/sota_annotations/dialogue_aggrefact.csv')
df_llm



,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
6,6,660f2253-fa69-4881-9666-6a0d69ef9a2c,ann_wjbp,13611370,gpt4-32k-0613,NaN,NaN,NaN,Derek asks Judy if she can feed his animals on...,"Derek: Judy, r you leaving for the weekend?\r\..."
7,7,650c80ed-35e6-4498-8314-6d8f0a85aa99,ann_wjbp,13611426,gpt4-32k-0613,"both prefer Lisbon, especially enjoying its ni...","Harry : Yeah me too, principally the night",Intrinsic_Error,"Poppy and Harry both prefer Lisbon, especially...",Poppy: Yeah. I definitely prefer Lisbon\r\nHar...
8,8,195b4835-e3a9-459b-acd5-6f4b22ed539f,ann_wjbp,13611442,gpt4-32k-0613,he was unavailable,I didn't hear you,Intrinsic_Error,"Helen was trying to reach Simon, but he was un...","Helen: Hey, Simo, are you there?\r\nSimon: Yep..."
9,9,02f228f5-435e-4906-bb98-1452bd51d708,ann_wjbp,13611508,gpt4-32k-0613,the products,I'll check them,Extrinsic_Error,"Martha, seeing Ophelia''s profile picture, is ...","Martha: Hey, can I ask you a question?\r\nOphe..."
10,10,741ca003-607f-4dab-9173-e780ea38d9a9,ann_wjbp,13611832,gpt4-32k-0613,NaN,NaN,NaN,Sam expresses concern that a guy doesn''t like...,Sam: I don''t think he likes me.\r\nCathy: Of ...
...,...,...,...,...,...,...,...,...,...,...
214,214,f03d7059-16ad-4f9e-84be-63c685247d8d,ann_wjbp,13731212,gpt4-32k-0613,NaN,NaN,NaN,Chandler asks Phoebe for money to pay for a de...,Chandler: Phoebe!! Do you have money?? \r\nPho...
215,215,bd253c24-6f0d-4e59-a9df-12692e83bcc0,ann_wjbp,13731225,gpt4-32k-0613,asks to wait for him to have dinner,Julia : I was wondering if we should wait for...,Intrinsic_Error,Bert tells Julia he will be home around 8 and ...,"Julia: Hey, what time are you getting home?\r\..."
216,216,eea85b65-f2d3-4b74-9ab8-b54dc7639fe5,ann_wjbp,13731261,gpt4-32k-0613,latest episode,Not found,Extrinsic_Error,Alec and Alexa discussed the latest episode of...,Alec: Have you seen the last America''s got ta...
217,217,979ada7f-2b74-4384-96f9-5cd1a4ce8b44,ann_wjbp,13731361,gpt4-32k-0613,NaN,NaN,NaN,Julia shares with Ronald that she is not enjoy...,Ronald: How''s your weekend?\r\nJulia: Fine I ...


In [17]:

import nltk
import re
import string 
import spacy 
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.corpus import stopwords
import numpy as np
from random import sample
from experiments.models.GPT.AtomicFacts import MakeAtomicFacts
from experiments.models.utils import process_gpt_response, make_masked_sentences

            

nltk.download('stopwords')
stop_words = stopwords.words('english')

nlp = spacy.load('en_core_web_sm')





    

class GPTPrompttInfill():
    def __init__(self):
        self.gpt_instructions = {
            'infill': '''Fill in the <BLANK> appropriately only using information found in the source. If there are no suitable options, respond with "None".''',
            'entailment': 'Decide if the premise entails the corresponding hypothesis.',
    
        }
        self.gpt_prompt_templates = {
            'infill': f"{self.gpt_instructions['infill']}\nSource: {{source}}\nSummary: {{summary}}",
            "infill_fewshot" : f"{self.gpt_instructions['infill']}\nSource: {{source}}\n{{fewshot_str}}\nSummary: {{summary}}\nAnswer:",
            'entailment': f"{self.gpt_instructions['entailment']}\nPremise: {{premise}}\nHypothesis: {{hypothesis}}\nRate on a scale of 1-5:"
        }
        
        self.gpt_models = {
            'infill': GPTInference(),
            'nli': GPTInference()
        }
        
        
    def check_entailment_score_gpt(self, premise, hypothesis):
        
        prompt = self.gpt_prompt_templates['entailment'].format(
                                       premise = premise,
                                       hypothesis = hypothesis)
        # print(prompt)
        answer = GPTInference().get_chatgpt_response(prompt)
        answer = re.findall(r'\d+', answer)
        if not answer:
            return 0
        else:
            answer = answer[0]
        return eval(answer)
    
    def phrase_similarity_infilled(self, summary_span, infill_answers, masked_sentence):
        summary_sentence = masked_sentence.replace('<BLANK>', summary_span)
    
        all_scores = []
        for infill in infill_answers:
            infill_sentence = masked_sentence.replace('<BLANK>', infill)
            entailment_score = self.check_entailment_score_gpt(premise = infill_sentence, hypothesis= summary_sentence)

            hyp_overlap = [each for each in summary_span.split(' ') if each.lower() in infill.lower()]
            perc_found = len(hyp_overlap)/len(summary_span.split(' '))

            print('SENT', masked_sentence)
            print('SPAN', summary_span)
            print('RECC', infill)
            print(perc_found, entailment_score)
            print('**')

            if (perc_found > 0.95) or (entailment_score > 4):
                return True
        # print('='*13)
        return False

    def predict_nonfactual_spans(self, dlg, summ, fewshot_num = 0):
        all_masked_results = make_masked_sentences(GPTInference(), summ)
        
        if fewshot_num:
            atomic_facts_dlg  = MakeAtomicFacts().get_atomic_facts_gpt(dlg)
            strat_sample = 1 if len(atomic_facts_dlg) >= fewshot_num else round(fewshot_num/len(atomic_facts_dlg))
            print('SENT SAMPLE', strat_sample)
            all_samples = []
            for af_idx, atomic_fact in enumerate(atomic_facts_dlg):
                masked_atomic_facts = make_masked_sentences(GPTInference(), atomic_fact)
                masked_samples = [(' '.join(atomic_facts_dlg[:af_idx] + [sent] + atomic_facts_dlg[af_idx+1:]), 
                                   sent, span) for _, sent, span in masked_atomic_facts]
                
                if len(masked_samples) > strat_sample:
                    masked_samples = sample(masked_samples, strat_sample)
                all_samples += masked_samples
            fewshot_str = [f'Summary: {summ}\nAnswer:{span}' for summ, _, span in all_samples[:fewshot_num]]
            fewshot_str = '\n'.join(fewshot_str)
            
        print_sample = 0
        answers = []
        nonfactual_sent_span = []
        for summary, sentence, answer in all_masked_results:   
            
            if fewshot_str: 
                prompt = self.gpt_prompt_templates['infill_fewshot'].format(source = dlg, fewshot_str = fewshot_str, summary = summary)
            else:
                prompt = self.gpt_prompt_templates['infill'].format(source = dlg, summary = summary)
                
            if print_sample < 1:
                print('PROMPT', prompt)
                # print_sample = False
                print_sample += 1
                
            gpt_response = self.gpt_models['infill'].get_chatgpt_response(prompt)
            response = process_gpt_response(gpt_response)

            if not self.phrase_similarity_infilled(answer, response, sentence):
                answers += [(answer, response)]
                # print('===')

        return answers
    
    

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ramprasad.sa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
from sentence_transformers import SentenceTransformer, util
class Evaluation:
    
    def __init__(self):
        self.match_found_threshold = 0.75 
        self.sim_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    def phrase_similarity(self, span1, span2):
        embedding_1= self.sim_model.encode(span1, convert_to_tensor=True)
        embedding_2 = self.sim_model.encode(span2, convert_to_tensor=True)

        sim = util.pytorch_cos_sim(embedding_1, embedding_2).tolist()[0][0]
        return sim

    
    def retrieve_sim_spans(self, ref_spans, comp_spans):
        found_spans = []
        for ref in ref_spans:
            matched_score = [self.phrase_similarity(ref, comp) for comp in comp_spans]
            matched_score = max(matched_score)
            if matched_score >= self.match_found_threshold:
                found_spans += [ref]
        return found_spans

    def token_f1_sim(self, annotated_spans, predicted_spans):
        matched_spans = self.retrieve_sim_spans(predicted_spans, annotated_spans)
        precision = 0
        recall = 0

        if not predicted_spans and not annotated_spans:
            return 1
        else:
            f1_score = 0
            if predicted_spans:
                precision = len(matched_spans)/len(predicted_spans)
            if annotated_spans:
                recall = len(matched_spans)/len(annotated_spans)

            if precision + recall > 0:
                f1_score = (2 * precision * recall)/(precision + recall)
                
            # print('PRECISION', precision)
            # print('RECALL', recall)
            return f1_score

    

        

In [19]:

zeroshot_gpt_infill = GPTPrompttInfill()

In [20]:
row_idx = 34
row = df_llm.iloc[[row_idx]]
summ = row['summary'].values[0]
dlg = row['dialogue'].values[0]

nonfactual_spans = list(df_llm[df_llm['dialogue'] == dlg]['nonfactual_spans'].values)
predicted_nonfactual_spans = zeroshot_gpt_infill.predict_nonfactual_spans(dlg, summ, 16)

SENT SAMPLE 2
PROMPT Fill in the <BLANK> appropriately only using information found in the source. If there are no suitable options, respond with "None".
Source: Joanna: They are sending emails about Lewandowska.
Merve: What happened?
Joanna: <file_photo> 
Merve: Wooow!
Joanna: She is hospitalized because she has measles.
Merve: She had what?
Joanna: Anyone who had contact with her within the last couple of days must get vaccinated.
Merve: Luckily I didn''t see her since the last semester...
Joanna: I did, she is my thesis mentor :(
Merve: What will you do?
Joanna: They are organizing vaccinations in the main building from 17th until 19th.
Merve: You have to go!
Joanna: I know... And I just started working so I really don''t have a lot of time.
Merve: Come on, this is really important.
Joanna: I will try to do it before work on 18th, hopefully I won''t lose the entire day...
Summary: they <BLANK> sending emails about lewandowska. Lewandowska is hospitalized. Lewandowska has measles. An

In [21]:


Evaluation().token_f1_sim(nonfactual_spans, predicted_nonfactual_spans)


0.5

In [75]:
nonfactual_spans

['The university', 'a new']

In [51]:
from random import sample

total_samples = 16
strat_sample = 1 if len(atomic_facts_dlg) >= 8 else round(8/len(atomic_facts_dlg))
print(strat_sample)

all_samples = []
for af_idx, atomic_fact in enumerate(atomic_facts_dlg):
    masked_atomic_facts = make_masked_sentences(atomic_fact)
    masked_samples = [(' '.join(atomic_facts_dlg[:af_idx] + [sent] + atomic_facts_dlg[af_idx+1:]), 
                       sent, span) for _, sent, span in masked_atomic_facts]
    # print(masked_samples)
    if len(masked_samples) > strat_sample:
        masked_samples = sample(masked_samples, strat_sample)
    all_samples += masked_samples
        
fewshot_str = [f'Summary: {summ}\nAnswer:{span}' for summ, _, span in all_samples[:total_samples]]
print('\n'.join(fewshot_str))

1
Summary: joanna received emails <BLANK> lewandowska. Lewandowska is hospitalized due to measles. Anyone who was in contact with Lewandowska recently has to get vaccinated. Merve hasn't seen Lewandowska since the last semester. Lewandowska is Joanna's thesis mentor. Vaccinations are being arranged in the main building from 17th to 19th. Joanna just started a new job and doesn't have much time. Joanna plans to get vaccinated on 18th before work.
Answer:about
Summary: Joanna received emails about Lewandowska. <BLANK> is hospitalized due to measles. Anyone who was in contact with Lewandowska recently has to get vaccinated. Merve hasn't seen Lewandowska since the last semester. Lewandowska is Joanna's thesis mentor. Vaccinations are being arranged in the main building from 17th to 19th. Joanna just started a new job and doesn't have much time. Joanna plans to get vaccinated on 18th before work.
Answer:lewandowska
Summary: Joanna received emails about Lewandowska. Lewandowska is hospitaliz

In [52]:
summ

'Lewandowska, who is a thesis mentor to Joanna, has been hospitalized due to measles. Joanna has received an email about this, also advising that anyone who has been in contact with Lewandowska in recent days must get vaccinated. The university is organizing vaccinations from the 17th to the 19th. Joanna, who just started a new job, plans to try and get the vaccine before work on the 18th.'

PROMPT Fill in the <BLANK> appropriately only using information mentioned in the source. If there are no suitable options, respond with "None".
Source: Joanna: They are sending emails about Lewandowska.
Merve: What happened?
Joanna: <file_photo> 
Merve: Wooow!
Joanna: She is hospitalized because she has measles.
Merve: She had what?
Joanna: Anyone who had contact with her within the last couple of days must get vaccinated.
Merve: Luckily I didn''t see her since the last semester...
Joanna: I did, she is my thesis mentor :(
Merve: What will you do?
Joanna: They are organizing vaccinations in the main building from 17th until 19th.
Merve: You have to go!
Joanna: I know... And I just started working so I really don''t have a lot of time.
Merve: Come on, this is really important.
Joanna: I will try to do it before work on 18th, hopefully I won''t lose the entire day...
Summary: joanna received emails <BLANK> lewandowska. Lewandowska is hospitalized due to measles. Anyone who was in contact

In [133]:
# all_masked_results
fewshot_str = [f'Summary: {sent}\nAnswer:{span}' for _, sent, span in all_masked_results[:4]]
print('\n'.join(fewshot_str))

Summary: richie brings up the topic of pogba clay refers to pogba as "pogboom richie expresses admiration for pogba's strike clay describes his anticipation seeing pogba’s skill on the field clay expresses hope that pogba's form will continue richie says that he sees pogba as <BLANK> this season according to clay, jose trusts pogba richie asserts that everyone trusts pogba clay believes pogba deserved to score after his first 0 minutes in the game 0
Answer:more mature
Summary: richie brings up the topic of pogba clay refers to pogba as "pogboom richie expresses admiration for pogba's strike clay describes his anticipation seeing pogba’s skill on the field clay expresses hope that pogba's form will continue richie says that he sees pogba as more mature this season according to clay, jose trusts pogba richie asserts that everyone trusts pogba clay believes pogba deserved to score after his first 0 minutes <BLANK> 0
Answer:in the game
Summary: richie brings up the topic of pogba clay refe

In [54]:
all_masked_results[0][1]

NameError: name 'all_masked_results' is not defined

In [30]:
# precision : Out of all in predicted what percentage was accurate
# recall; Out of all in truth how many were extracted

match_found_threshold = 0.75 

def phrase_similarity(span1, span2):
    embedding_1= model.encode(span1, convert_to_tensor=True)
    embedding_2 = model.encode(span2, convert_to_tensor=True)

    sim = util.pytorch_cos_sim(embedding_1, embedding_2).tolist()[0][0]
    return sim



def retrieve_sim_spans(ref_spans, comp_spans):
    found_spans = []
    for ref in ref_spans:
        matched_score = [phrase_similarity(ref, comp) for comp in comp_spans]
        matched_score = max(matched_score)
        if matched_score >= match_found_threshold:
            found_spans += [ref]
    return found_spans

def token_f1_sim(annotated_spans, predicted_spans):
    matched_spans = retrieve_sim_spans(predicted_spans, annotated_spans)
    precision = 0
    recall = 0
    
    if not predicted_spans and not annotated_spans:
        return 1
    else:
        f1_score = 0
        if predicted_spans:
            precision = len(matched_spans)/len(predicted_spans)
        if annotated_spans:
            recall = len(matched_spans)/len(annotated_spans)
        
        if precision + recall > 0:
            f1_score = (2 * precision * recall)/(precision + recall)
        return f1_score
    
    
token_f1_sim(nonfactual_spans, predicted_nonfactual_spans)
        
        

0

In [146]:

Input: Dialogue/nSUmmary: <BLANK> is a player 
Output: Answer: Pogba

NameError: name 'Dialogue' is not defined

In [24]:
from sentence_transformers import SentenceTransformer, util
sentences = ["I'm happy", "I'm full of happiness"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_1= model.encode(predicted_nonfactual_spans[2], convert_to_tensor=True)
embedding_2 = model.encode(nonfactual_spans[0], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2).tolist()[0][0]
## tensor([[0.6003]])


0.36964523792266846

In [1]:
from experiments.models.Mistral import MistralInference
prompt = "Hey, are you conscious? Can you talk to me?"
MistralInference().get_response(prompt)

/home/ramprasad.sa/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.73s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Hey, are you conscious? Can you talk to me?\n\nI’m not sure if you’re conscious or not. I’m not sure if you can talk to me.\n\nI’m not sure if you’re conscious or not. I’m not sure if you can talk to me.\n\nI’m not sure if you’re conscious or not. I’m not sure if you can talk to me.\n\nI’m not sure if you’re conscious or not. I’m not sure if you can talk to me.\n\nI’m not sure if you’re conscious or not. I’m not sure if you can talk to me.\n\nI’m not sure if you’re conscious or not. I’m not sure if you can talk to me.\n\nI’m not sure if you’re conscious or not. I’m not sure if you can talk to'

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir='/scratch/ramprasad.sa/huggingface_models/')
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir='/scratch/ramprasad.sa/huggingface_models/')

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.35s/it]


In [6]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")
device = "cuda"
model = model.to(device)
input_ids = inputs.input_ids.to(device)
generate_ids = model.generate(input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Hey, are you conscious? Can you talk to me?\n\nI’m not sure if you’re conscious or not. I’'

In [47]:
instruction = '''Extract important noun and verb phrases from the sentence. List each in a new line'''
sent = sents[0]
prompt = f'{instruction}\nSentence:{sent}\nSpans:'
response = GPTInference().get_chatgpt_response(prompt)
spans = response.split('\n')
summ_sent_mask_infill = []
for span in spans:
    masked_sent = sent.replace(span, '<BLANK>')
    summ_sent_mask_infill.append((masked_sent, span))

In [48]:
summ_sent_mask_infill

[('<BLANK> and Deborah are planning a trip with their girls.', 'Nathan'),
 ('Nathan and <BLANK> are planning a trip with their girls.', 'Deborah'),
 ('Nathan and Deborah are <BLANK> a trip with their girls.', 'planning'),
 ('Nathan and Deborah are planning a <BLANK> with their girls.', 'trip'),
 ('Nathan and Deborah are planning a trip with <BLANK>.', 'their girls')]

In [16]:
make_masked_sentences(summ)

[('<BLANK> and Deborah are planning a trip with their girls.', 'Nathan'),
 ('Nathan and <BLANK> are planning a trip with their girls.', 'Deborah'),
 ('Nathan and Deborah are <BLANK> a trip with their girls.', 'planning'),
 ('Nathan and Deborah are planning a <BLANK> with their girls.', 'trip'),
 ('Nathan and Deborah are planning a trip with <BLANK>.', 'their girls'),
 ("<BLANK> to pack everything today, including boots and warm jackets for their daughters since they will go on a forest trip with Deborah''s parents.",
  'They agreed'),
 ("They agreed to <BLANK> today, including boots and warm jackets for their daughters since they will go on a forest trip with Deborah''s parents.",
  'pack everything'),
 ("They agreed to pack everything <BLANK>, including boots and warm jackets for their daughters since they will go on a forest trip with Deborah''s parents.",
  'today'),
 ("They agreed to pack everything today, <BLANK> and warm jackets for their daughters since they will go on a forest 

In [ ]:



# gpt_model_nli = GPTInference()
# gpt_model_afacts = GPTInference()
# gpt_model_infill = GPTInference()

# def get_atomic_facts_gpt( text):
#     instr = f'Segment the following dialogue into important facts without introducing any unsupported information'
#     prompt = f'{instr}\nDialogue: {text}'
#     # print(prompt)
#     gpt_response = gpt_model_afacts.get_chatgpt_response(prompt)
#     # print(nltk.sent_tokenize(gpt_response))
#     return gpt_response
    
# def get_atomic_facts( dlg):
#         atomic_facts = get_atomic_facts_gpt( dlg,)
#         atomic_facts = nltk.sent_tokenize(atomic_facts)
#         atomic_facts = [re.sub('[1-9]', '', each) for each in atomic_facts]
#         atomic_facts = [each.strip(string.punctuation).strip() for each in atomic_facts]
#         return atomic_facts
    


# def check_entailment_score(premise, hypothesis):
#     instruction = 'Decide if the premise entails the corresponding hypothesis.' 
#     # Note that consistency means all information in the hypothesis is supported by the premise'
#     prompt_template = f'{instruction}\nPremise: {{premise}}\nHypothesis: {{hypothesis}}\nRate on a scale of 1-10:'
#     prompt = prompt_template.format(
#                                    premise = premise,
#                                    hypothesis = hypothesis)
#     answer = gpt_model_nli.get_chatgpt_response(prompt)
#     answer = re.findall(r'\d+', answer)
#     if not answer:
#         return 0

#     else:
#         answer = answer[0]
    
#     return eval(answer)



    
    

    
# def get_nonfactual_comparators(all_masked_results, dlg):
#     instruction = '''List a words or phrases that can fill in the <BLANK> in the sentence based on information extracted from the dialogue. Ensure the sentence is consistent with the source where consistency means all information in the sentence is found in the source. If there are no suitable options, respond with "None.'''
    
#     prompt_template = f"{{instruction}}:\\nSource: {{source}}\nSentence: {{sentence}}"
    
#     answers = []
#     nonfactual_sent_span = []
#     for summary, sentence, answer in all_masked_results:   
#         prompt = prompt_template.format(instruction = instruction, source = dlg, sentence = summary)
#         gpt_response = gpt_model_infill.get_chatgpt_response(prompt)
#         response = process_gpt_response(gpt_response)
        
#         if not phrase_similarity(answer, response, sentence)  :
#             # print(sentence, answer, response)
#             answers += [(answer, response)]
#             # print('===')
        
#     return answers